In [ ]:
import pandas as pd; pd.set_option('display.max_columns', None)
import time
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium import webdriver
from  selenium.webdriver.common.keys import Keys
from datetime import date
import numpy as np

## Atualização da temporada

In [ ]:
url = 'https://www.flashscore.com.br/beisebol/eua/mlb/resultados/'
temp_path = 'data/mlb/mlb-2023.csv'
n_jogos = 17

In [ ]:
# Instanciando o Objeto ChromeOptions
options = webdriver.EdgeOptions()
 
options.add_argument('--headless')
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage') 
 
wd_Chrome = webdriver.Edge(options=options)

# Com o WebDrive a gente consegue a pedir a página (URL)
wd_Chrome.get(url)

time.sleep(1)

# Apertando o botão de carregar mais jogos
if n_jogos > 100:
    try:
        loadMore = wd_Chrome.find_element(By.CSS_SELECTOR, 'a.event__more--static')
        isPresent = loadMore.is_displayed()
        cliquei = 0
    
        while isPresent:
            try:
                cliquei = cliquei + 1
                print(f'Cliquei Load More {cliquei} vezes')
                wd_Chrome.find_element(By.CSS_SELECTOR, 'a.event__more--static').send_keys(Keys.ENTER)
                time.sleep(2)
                isPresent = loadMore.is_displayed()
            except:
                isPresent = False
                break
    except:
        print("LoadMore não encontrado na página.")
        pass

# Pegando o ID dos Jogos
id_jogos = []
jogos = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.event__match--static')

for i in jogos:
    id_jogos.append(i.get_attribute("id"))

# Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
id_jogos = [i[4:] for i in id_jogos]

# Pegando apenas os últimos n_jogos
id_jogos = id_jogos[:n_jogos]

base_jogos = pd.DataFrame({})

for link in tqdm(id_jogos, total=len(id_jogos)):
    wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/resumo-de-jogo/resumo-de-jogo')
    
    Odds_H = 0
    Odds_A = 0
    Over_Line = 0
    Odds_Over = 0
    Odds_Under = 0
    HA_Line = 0
    HA_Odds_H = 0
    HA_Odds_A = 0
    
    # Pegando as Informacoes Básicas do Jogo
    try:
        Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        Season = Date.split('.')[2]
        League = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country')
        League = League.find_element(By.CSS_SELECTOR,'a').text
        Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Home_Pts = wd_Chrome.find_element(By.CSS_SELECTOR,'div.detailScore__wrapper').text.split('-')[0]
        Away_Pts = wd_Chrome.find_element(By.CSS_SELECTOR,'div.detailScore__wrapper').text.split('-')[1]
        
        # Match Odds
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/home-away/tr-incluindo-prol')
        time.sleep(0.5)
        celulas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for celula in celulas:
            bookie = celula.find_element(By.CSS_SELECTOR,'img.prematchLogo')
            bookie = bookie.get_attribute('title')
            if (((bookie == 'bet365') and (Odds_H == 0)) or ((bookie == '1xBet') and (Odds_H == 0))):
                Odds_H = float(celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                Odds_A = float(celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
            else:
                pass


        # Over/Under
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/acima-abaixo/tr-incluindo-prol')
        time.sleep(0.5)
        celulas = wd_Chrome.find_elements(By.CLASS_NAME,'ui-table__row')


        for celula in celulas:
            bookie = celula.find_element(By.CSS_SELECTOR, 'img.prematchLogo').get_attribute('title')
            Odds_Over = float(celula.find_elements(By.TAG_NAME,'span')[1].text)
            if (((bookie == 'bet365') and (Odds_Over >= 1.80)) or ((bookie == '1xBet') and (Odds_Over >= 1.80))):
                Over_Line = float(celula.find_elements(By.TAG_NAME,'span')[0].text)
                Odds_Under = float(celula.find_elements(By.TAG_NAME,'span')[2].text)
                break
            else:
                pass


        # Handicap
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/handicap-asiatico/tr-incluindo-prol')
        time.sleep(0.5)
        celulas = wd_Chrome.find_elements(By.CLASS_NAME,'ui-table__row')


        for celula in celulas:

            bookie = celula.find_element(By.CSS_SELECTOR, 'img.prematchLogo').get_attribute('title')
            HA_Odds_H = float(celula.find_elements(By.TAG_NAME,'span')[1].text)
            if (((bookie == 'bet365') and (HA_Odds_H <=2.8)) or ((bookie == '1xBet') and (HA_Odds_H <= 2.8))):
                HA_Line = float(celula.find_elements(By.TAG_NAME,'span')[0].text)
                HA_Odds_A = float(celula.find_elements(By.TAG_NAME,'span')[2].text)
                break
            else:
                pass

        base_jogos.loc[base_jogos.shape[0],['Date','Season','Season_Time','Time','Home','Away','Home_Pts','Away_Pts','Odds_H','Odds_A','Over_Line','Odds_Over','Odds_Under','HA_Line','HA_Odds_H','HA_Odds_A']] = [
            Date, Season, League, Time, Home, Away, Home_Pts, Away_Pts, Odds_H, Odds_A, Over_Line, Odds_Over, Odds_Under, HA_Line, HA_Odds_H, HA_Odds_A
        ]
    except:
        continue

wd_Chrome.quit()

colunas_float = ['Home_Pts', 'Away_Pts', 'Odds_H', 'Odds_A', 'Over_Line',
    'Odds_Over', 'Odds_Under', 'HA_Line', 'HA_Odds_H', 'HA_Odds_A']
base_jogos[colunas_float] = base_jogos[colunas_float].astype(float)

# Ajusta coluna de data
base_jogos['Date'] = pd.to_datetime(base_jogos['Date'], format='%d.%m.%Y')
base_jogos = base_jogos.sort_values(by='Date')
base_jogos.reset_index(drop=True, inplace=True)

In [ ]:
temporada = pd.read_csv(temp_path)
temporada['Date'] = pd.to_datetime(temporada['Date'])

# Exclui os registros que já existem no dataset da temporada
temporada = pd.concat([temporada, base_jogos], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)

In [ ]:
temporada.to_csv(temp_path, index=False)

## Scraping dos jogos do dia

In [ ]:
# Instanciando o Objeto ChromeOptions
options = webdriver.EdgeOptions()

# Passando algumas opções para esse ChromeOptions
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Criação do WebDriver do Chrome
wd_Chrome = webdriver.Edge(options=options)

# Com o WebDrive a gente consegue a pedir a página (URL)
wd_Chrome.get("https://www.flashscore.com.br/beisebol/eua/mlb/#/4O099Es6/table/overall")
time.sleep(2)

# Para jogos do dia
jogos = wd_Chrome.find_element(By.CSS_SELECTOR, 'div.leagues--live')

# Pegando o ID dos Jogos
id_jogos = []
jogos = jogos.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')

for i in jogos:
    id_jogos.append(i.get_attribute("id"))

# Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
id_jogos = [i[4:] for i in id_jogos]

base_jogos = pd.DataFrame({})

for link in tqdm(id_jogos, total=len(id_jogos)):
    wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/resumo-de-jogo')
    
    Odds_H = 0
    Odds_A = 0
    Over_Line = 0
    Odds_Over = 0
    Odds_Under = 0
    HA_Line = 0
    HA_Odds_H = 0
    HA_Odds_A = 0

    # Pegando as Informacoes Básicas do Jogo
    try:
        Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        Season = Date.split('.')[2]
        Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        
        # Match Odds
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/home-away/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for celula in celulas:
            bookie = celula.find_element(By.CSS_SELECTOR,'img.prematchLogo')
            bookie = bookie.get_attribute('title')
            if (((bookie == 'bet365') and (Odds_H == 0)) or ((bookie == '1xBet') and (Odds_H == 0))):
                Odds_H = float(celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                Odds_A = float(celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
            else:
                pass


        # Over/Under
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/acima-abaixo/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CLASS_NAME,'ui-table__row')


        for celula in celulas:
            bookie = celula.find_element(By.CSS_SELECTOR, 'img.prematchLogo').get_attribute('title')
            Odds_Over = float(celula.find_elements(By.TAG_NAME,'span')[1].text)
            if (((bookie == 'bet365') and (Odds_Over >= 1.80)) or ((bookie == '1xBet') and (Odds_Over >= 1.80))):
                Over_Line = float(celula.find_elements(By.TAG_NAME,'span')[0].text)
                Odds_Under = float(celula.find_elements(By.TAG_NAME,'span')[2].text)
                break
            else:
                pass


        # Handicap
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/handicap-asiatico/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CLASS_NAME,'ui-table__row')


        for celula in celulas:

            bookie = celula.find_element(By.CSS_SELECTOR, 'img.prematchLogo').get_attribute('title')
            HA_Odds_H = float(celula.find_elements(By.TAG_NAME,'span')[1].text)
            if (((bookie == 'bet365') and (HA_Odds_H <=2.8)) or ((bookie == '1xBet') and (HA_Odds_H <= 2.8))):
                HA_Line = float(celula.find_elements(By.TAG_NAME,'span')[0].text)
                HA_Odds_A = float(celula.find_elements(By.TAG_NAME,'span')[2].text)
                break
            else:
                pass

    except:
        pass

    base_jogos.loc[base_jogos.shape[0],['Date','Season','Time','Home','Away','Odds_H','Odds_A','Over_Line','Odds_Over','Odds_Under','HA_Line','HA_Odds_H','HA_Odds_A']] = [
        Date, Season, Time, Home, Away, Odds_H, Odds_A, Over_Line, Odds_Over, Odds_Under, HA_Line, HA_Odds_H, HA_Odds_A]

wd_Chrome.quit()
base_jogos.drop(base_jogos[(base_jogos == 0).any(axis=1)].index, inplace=True)

In [ ]:
# base_jogos.to_csv("~/Desktop/MLB-jogos_do_dia.csv", index=False)
base_jogos

## Apostas do dia

#### Funções

In [ ]:
################################################################################
# Prepara o DF
################################################################################

def prepara_df(df):
  df = df[(df.Over_Line >= 5)]
  df = df[(df['Home_Pts'] + df['Away_Pts']) != df['Over_Line']]
  df = df[df['HA_Odds_A'] != 0]

  df['P(H)'] = 1 / df['Odds_H']
  df['P(A)'] = 1 / df['Odds_A']
  df['P(O)'] = 1 / df['Odds_Over']
  df['P(U)'] = 1 / df['Odds_Under']
  
  df.loc[(df['Home_Pts'] + df['Away_Pts']) > df['Over_Line'], 'Back_Over'] = 1
  df.loc[(df['Home_Pts'] + df['Away_Pts']) < df['Over_Line'], 'Back_Over'] = 0

  df['P_Diff'] = ((1 / df['Odds_H']) + (1 / df['Odds_A'])) - 1

  df['Porc_Over_Home'] = df.groupby('Home')['Back_Over'].rolling(3).mean().reset_index(level=0, drop=True)
  df['Porc_Over_Away'] = df.groupby('Away')['Back_Over'].rolling(3).mean().reset_index(level=0, drop=True)
  df['Porc_Over_Home']  = df.groupby('Home')['Porc_Over_Home'].shift(1)
  df['Porc_Over_Away']  = df.groupby('Away')['Porc_Over_Away'].shift(1)
  df['Porc_Over_Home'] = df['Porc_Over_Home'].replace(np.nan, 0)
  df['Porc_Over_Away'] = df['Porc_Over_Away'].replace(np.nan, 0)
  df.reset_index(inplace=True, drop=True)

  # Custo do gol
  df['CustoGolHome'] = df['Home_Pts'] / (1 / df['Odds_H'])
  df['CustoGolAway'] = df['Away_Pts'] / (1 / df['Odds_A'])
  df['CustoGolHome'] = df['CustoGolHome'].replace(np.inf, 0)
  df['CustoGolAway'] = df['CustoGolAway'].replace(np.inf, 0)
  df.reset_index(drop=True, inplace=True)

  # Último custo do gol
  df['Last_CG_H']  = df.groupby('Home')['CustoGolHome'].shift(1)
  df['Last_CG_A']  = df.groupby('Away')['CustoGolAway'].shift(1)
  df['Last_CG_H'] = df['Last_CG_H'].replace(np.nan, 0)
  df['Last_CG_A'] = df['Last_CG_A'].replace(np.nan, 0)

  # Média móvel do custo do gol
  df['MediaCustoGolHome'] = df.groupby('Home')['CustoGolHome'].rolling(window=3).mean().reset_index(level=0, drop=True)
  df['MediaCustoGolAway'] = df.groupby('Away')['CustoGolAway'].rolling(window=3).mean().reset_index(level=0, drop=True)

  df['MediaCustoGolHome']  = df.groupby('Home')['MediaCustoGolHome'].shift(1)
  df['MediaCustoGolAway']  = df.groupby('Away')['MediaCustoGolAway'].shift(1)

  df['MediaCustoGolHome'] = df['MediaCustoGolHome'].replace(np.nan, 0)
  df['MediaCustoGolAway'] = df['MediaCustoGolAway'].replace(np.nan, 0)

  limit_up_h = df.CustoGolHome.mean() + df.CustoGolHome.std()
  limit_up_a = df.CustoGolAway.mean() + df.CustoGolAway.std()
  df.loc[(df['CustoGolHome'] > limit_up_h), 'Acima_Last_CG_H'] = 1
  df.loc[(df['CustoGolHome'] <= limit_up_h), 'Acima_Last_CG_H'] = 0
  df.loc[(df['CustoGolAway'] > limit_up_a), 'Acima_Last_CG_A'] = 1
  df.loc[(df['CustoGolAway'] <= limit_up_a), 'Acima_Last_CG_A'] = 0
  df['Acima_Last_CG_H']  = df.groupby('Home')['Acima_Last_CG_H'].shift(1)
  df['Acima_Last_CG_A']  = df.groupby('Away')['Acima_Last_CG_A'].shift(1)
  df['Acima_Last_CG_H'] = df['Acima_Last_CG_H'].replace(np.nan, 0)
  df['Acima_Last_CG_A'] = df['Acima_Last_CG_A'].replace(np.nan, 0)

  limit_down_h = df.CustoGolHome.mean() - df.CustoGolHome.std()
  limit_down_a = df.CustoGolAway.mean() - df.CustoGolAway.std()
  df.loc[(df['CustoGolHome'] < limit_down_h), 'Abaixo_Last_CG_H'] = 1
  df.loc[(df['CustoGolHome'] >= limit_down_h), 'Abaixo_Last_CG_H'] = 0
  df.loc[(df['CustoGolAway'] < limit_down_a), 'Abaixo_Last_CG_A'] = 1
  df.loc[(df['CustoGolAway'] >= limit_down_a), 'Abaixo_Last_CG_A'] = 0
  df['Abaixo_Last_CG_H']  = df.groupby('Home')['Abaixo_Last_CG_H'].shift(1)
  df['Abaixo_Last_CG_A']  = df.groupby('Away')['Abaixo_Last_CG_A'].shift(1)
  df['Abaixo_Last_CG_H'] = df['Abaixo_Last_CG_H'].replace(np.nan, 0)
  df['Abaixo_Last_CG_A'] = df['Abaixo_Last_CG_A'].replace(np.nan, 0)

  df['CV_ML'] = (df[['Odds_H', 'Odds_A']].std(axis=1)) / (df[['Odds_H', 'Odds_A']].mean(axis=1))
  df['CV_Over'] = (df[['Odds_Over', 'Odds_Under']].std(axis=1)) / (df[['Odds_Over', 'Odds_Under']].mean(axis=1))
  
  return df

In [ ]:
def atualiza_dia(jogos_hoje, temporada):
    jogos_hoje['P(H)'] = 1 / jogos_hoje['Odds_H']
    jogos_hoje['P(A)'] = 1 / jogos_hoje['Odds_A']
    jogos_hoje['P(O)'] = 1 / jogos_hoje['Odds_Over']
    jogos_hoje['P(U)'] = 1 / jogos_hoje['Odds_Under']
    jogos_hoje['CV_ML'] = (jogos_hoje[['Odds_H', 'Odds_A']].std(axis=1)) / (jogos_hoje[['Odds_H', 'Odds_A']].mean(axis=1))
    jogos_hoje['CV_Over'] = (jogos_hoje[['Odds_Over', 'Odds_Under']].std(axis=1)) / (jogos_hoje[['Odds_Over', 'Odds_Under']].mean(axis=1))
    jogos_hoje['P_Diff'] = ((1 / jogos_hoje['Odds_H']) + (1 / jogos_hoje['Odds_A'])) - 1

    temporada = prepara_df(temporada)

    df_home = temporada.groupby('Home').last()[['Porc_Over_Home', 'MediaCustoGolHome', 'Acima_Last_CG_H', 'Abaixo_Last_CG_H', 'CustoGolHome', 'Last_CG_H']]
    df_away = temporada.groupby('Away').last()[['Porc_Over_Away', 'MediaCustoGolAway', 'Acima_Last_CG_A', 'Abaixo_Last_CG_A', 'CustoGolAway', 'Last_CG_A']]

    jogos_hoje = pd.merge(jogos_hoje, df_home, on='Home', how='left')
    jogos_hoje = pd.merge(jogos_hoje, df_away, on='Away', how='left')

    jogos_hoje.rename(columns={'CustoGolHome':'Last_CG_Home', 'CustoGolAway':'Last_CG_Away'}, inplace=True)

    jogos_hoje = jogos_hoje.reset_index(drop=True)

    return jogos_hoje

In [ ]:
df = base_jogos.copy()
df = atualiza_dia(df, temporada)

#### Jogos Método Over de hoje

In [ ]:
# Duplicando o dataset para Over
df_over = df.copy()

df_over = df_over[((df_over['Last_CG_Home'] >= 0) & (df_over['Last_CG_Home'] < 7)) | ((df_over['Last_CG_Home'] > 8) & (df_over['Last_CG_Home'] < 12))]
df_over = df_over[(df_over['CV_Over'] > 0.01)]
df_over = df_over[((df_over['MediaCustoGolAway'] > 2) & (df_over['MediaCustoGolAway'] < 7)) | ((df_over['MediaCustoGolAway'] > 12) & (df_over['MediaCustoGolAway'] < 17))]
df_over = df_over[(df_over['P(H)'] > 0.2) & (df_over['P(H)'] < 0.6)]
df_over = df_over[(df_over['P_Diff'] > 0.032) & (df_over['P_Diff'] < 0.062)]

df_over_conferencia = df_over.copy()

df_over['Bet'] = 'Over'
df_over = df_over[['Time', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'Bet']]
df_over.sort_values('Time', inplace=True)
df_over.reset_index(drop=True, inplace=True)

hoje = date.today()
# df_over.to_excel(f'~/Desktop/MLB_over_{hoje}.xlsx', index=False)

print(f'{len(df_over)} jogos para o método Over de hoje ({hoje})')
df_over

#### Jogos Método Under de hoje

In [ ]:
# Duplicando o dataset para Under
df_under = df.copy()

df_under = df_under[(df_under['P(O)'] > 0.524) & (df_under['P(O)'] < 0.554)]
df_under = df_under[(df_under['MediaCustoGolHome'] > 6) & (df_under['MediaCustoGolHome'] < 16)]
df_under = df_under[(df_under['CV_ML'] < 0.4)]
df_under = df_under[(df_under['Abaixo_Last_CG_A'] == 0)]
df_under = df_under[((df_under['Last_CG_H'] > 1) & (df_under['Last_CG_H'] < 4)) | ((df_under['Last_CG_H'] > 5) & (df_under['Last_CG_H'] < 7)) | ((df_under['Last_CG_H'] > 10) & (df_under['Last_CG_H'] < 12))]

df_under_conferencia = df_under.copy()

df_under['Bet'] = 'Under'
df_under = df_under[['Time', 'Home', 'Away', 'Over_Line', 'Odds_Under', 'Bet']]
df_under.sort_values('Time', inplace=True)
df_under.reset_index(drop=True, inplace=True)

hoje = date.today()
# df_under.to_excel(f'~/Desktop/MLB_over_{hoje}.xlsx', index=False)

print(f'{len(df_under)} jogos para o método Under de hoje ({hoje})')
df_under

#### Jogos Método Handicap Away de hoje

In [ ]:
# Duplicando o dataset para Handicap Away
df_HA_A = df.copy()

df_HA_A = df_HA_A[(df_HA_A['HA_Line'] < 0)]
df_HA_A = df_HA_A[(df_HA_A['P_Diff'] > 0.032) & (df_HA_A['P_Diff'] < 0.052)]
df_HA_A = df_HA_A[(df_HA_A['P(O)'] > 0.504)]
df_HA_A = df_HA_A[(df_HA_A['Acima_Last_CG_H'] == 0)]
df_HA_A = df_HA_A[(df_HA_A['CV_ML'] < 0.4)]
df_HA_A = df_HA_A[(df_HA_A['Last_CG_A'] > 4) & (df_HA_A['Last_CG_A'] < 12)]

df_HA_A_conferencia = df_HA_A.copy()

df_HA_A['Bet'] = 'HA Away'
df_HA_A = df_HA_A[['Time', 'Home', 'Away', 'HA_Line', 'HA_Odds_A', 'Bet']]
df_HA_A.sort_values('Time', inplace=True)
df_HA_A.reset_index(drop=True, inplace=True)

hoje = date.today()
# df_HA_A.to_excel(f'~/Desktop/MLB_over_{hoje}.xlsx', index=False)

print(f'{len(df_HA_A)} jogos para o método Handicap Away de hoje ({hoje})')
df_HA_A